In [2]:
#pip3 install twilio
#pip install bs4
#pip install tqdm

In [3]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [4]:
print("Account SID", TWILIO_ACCOUNT_SID)

print("Token", TWILIO_AUTH_TOKEN)

Account SID ACe98cc22418d6aa9fab461892dffd38c3
Token 9ba08fbf2af24974260a98932d2090bb


# Armado de la URL

## Call

In [5]:
#Preparando Llamada o url 
query = 'Ciudad Juarez'
api_key = API_KEY_WAPI
url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=2&aqi=no&alerts=no'

In [6]:
#Realizando llamada o peticion y conviertiendo resultado en formato JSON
response = requests.get(url_clima).json()

In [7]:
#Mostrando resultado
response

{'location': {'name': 'Ciudad Juarez',
  'region': 'Chihuahua',
  'country': 'Mexico',
  'lat': 31.73,
  'lon': -106.48,
  'tz_id': 'America/Ojinaga',
  'localtime_epoch': 1676331096,
  'localtime': '2023-02-13 17:31'},
 'current': {'last_updated_epoch': 1676330100,
  'last_updated': '2023-02-13 17:15',
  'temp_c': 10.0,
  'temp_f': 50.0,
  'is_day': 1,
  'condition': {'text': 'Light rain',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png',
   'code': 1183},
  'wind_mph': 18.6,
  'wind_kph': 29.9,
  'wind_degree': 270,
  'wind_dir': 'W',
  'pressure_mb': 1009.0,
  'pressure_in': 29.8,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 50,
  'cloud': 100,
  'feelslike_c': 5.9,
  'feelslike_f': 42.6,
  'vis_km': 16.0,
  'vis_miles': 9.0,
  'uv': 4.0,
  'gust_mph': 29.5,
  'gust_kph': 47.5},
 'forecast': {'forecastday': [{'date': '2023-02-13',
    'date_epoch': 1676246400,
    'day': {'maxtemp_c': 21.6,
     'maxtemp_f': 70.9,
     'mintemp_c': 4.1,
     'mintemp_f': 39.4,
  

In [8]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [9]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [10]:
#viendo la longitud de la hora
len(response['forecast']['forecastday'][0]['hour'])

24

In [11]:
#Obteniendo la fecha
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-02-13'

In [12]:
#Obteniendo Hora
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(":")[0])

1

In [13]:
#Condicion
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Partly cloudy'

In [14]:
#Temperatura
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

11.6

In [15]:
#Llovera? 0 para no, 1 para si
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [16]:
#posiblidad de lluvia
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

Dataframe

In [17]:
def get_forecast(response, i):
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(":")[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperatura = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    lluvia = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_lluvia = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return fecha, hora, condicion, temperatura, lluvia, prob_lluvia


In [28]:
datos = []

#libreria tqdm permite visualizar una barra de carga
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = "green"):
    datos.append(get_forecast(response, i))

100%|██████████| 24/24 [00:00<00:00, 24221.20it/s]


In [19]:
#Visualizando primer registro
datos[0]

('2023-02-13', 0, 'Partly cloudy', 11.6, 0, 0)

In [20]:
#Creando data Frame
col = ['fecha', 'hora', 'condicion', 'temperatura', 'lluvia', 'prob_lluvia']
df = pd.DataFrame(datos, columns=col)
df

,fecha,hora,condicion,temperatura,lluvia,prob_lluvia
0,2023-02-13,0,Partly cloudy,11.6,0,0
1,2023-02-13,1,Cloudy,10.5,0,0
2,2023-02-13,2,Clear,9.6,0,0
3,2023-02-13,3,Clear,8.6,0,0
4,2023-02-13,4,Clear,7.8,0,0
5,2023-02-13,5,Clear,6.9,0,0
6,2023-02-13,6,Clear,6.0,0,0
7,2023-02-13,7,Sunny,5.1,0,0
8,2023-02-13,8,Sunny,4.6,0,0
9,2023-02-13,9,Sunny,4.1,0,0


In [21]:
df_rain = df[ (df["lluvia"]==1) & (df["hora"]>6) & (df["hora"]<24)]
#df_rain = df[ (df["hora"]>6) & (df["hora"]<24)]
df_rain = df_rain[["hora", "condicion"]]
df_rain.set_index("hora", inplace = True)

In [22]:
df_rain

,condicion
hora,


In [23]:
df["fecha"][0]

'2023-02-13'

Mensaje - Template

In [24]:
if df_rain.empty:
    mensaje = '\nHola \n\n\n El pronostico del tiempo hoy '+ df['fecha'][0] + ' en ' + query + ' es: \n\n\n' + "Sin posibilidad de lluvia"
else:
    mensaje = '\nHola \n\n\n El pronostico del tiempo hoy '+ df['fecha'][0] + ' en ' + query + ' es: \n\n\n' + str(df_rain)

In [25]:
mensaje

'\nHola \n\n\n El pronostico del tiempo hoy 2023-02-13 en Ciudad Juarez es: \n\n\nSin posibilidad de lluvia'

In [26]:
PHONE_NUMBER

'+1 385 462 8057'

Mensaje SMS desde Twilio

In [27]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = TWILIO_ACCOUNT_SID
auth_token = "92cd3b4491a17776c413d761807286db" #TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
         body= mensaje,
         from_=PHONE_NUMBER,
         to='+526565656768'
         #to='+526562665660'
     )

print('Mensaje Enviado' + message.sid)

Mensaje EnviadoSMe8572d41b866bd2e7dfc496610255cf7
